# Spatial transcriptomics exercise

Please hand in the completed ipynb and a html or pdf version of the notebook by email to daria.romanovskaia@helmholtz-munich.de by **27.01.2026**.

The exercise was kindly provided by louis.kummerle@helmholtz-munich.de

For any qustions please contact the tutors of the seminar (Daria Romanovskaia, Benedikt Roth, Sarah Ouologuem, Xiatong Fu) on Slack.

You will get familiar to spatial transcriptomics analyses by looking at data from the human heart.

There is a total of 50 points that can be achieved.

## Installing and running
The exercise can be implemented in this google colab notebook. If you want to work on colab, please create a copy of the notebook. Alternatively you can download the ipynb notebook and work on your local jupyter server. For the local setup to work, please install python 3.10 and jupyter on your laptop.

In both cases you need to install the squidpy package.

In [ ]:
!pip install squidpy

## Import packages

In [ ]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
import zipfile

## for downloading
from pathlib import Path
from urllib.request import urlretrieve

## Exercise 1: deconvolution of spot based data from the human heart

The data for this exercise comes from the version 2 of the human heart cell atlas published in this paper:

*Spatially resolved multiomics of human cardiac niches*
* https://www.nature.com/articles/s41586-023-06311-1
* https://www.heartcellatlas.org/


In the paper, the authors characterized the cells of the cardiac conduction system. The data was obtained using the spot based Visium platform on the one hand and using single nucleus RNA-seq.

**Aim of this exercise: understand and reproduce figure 1 c-f of the paper**

![image on the nature website](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41586-023-06311-1/MediaObjects/41586_2023_6311_Fig1_HTML.png?as=webp)

Download the exercise data

In [ ]:
DATA_DIR = Path("./data")
DATA_DIR.mkdir(exist_ok=True, parents=True)

urls = [
    # Ex 1.2
    "https://cellgeni.cog.sanger.ac.uk/heartcellatlas/v2/SAN_aCMs_lognormalised.h5ad",
    "https://cellgeni.cog.sanger.ac.uk/heartcellatlas/v2/AVN_aCMs_lognormalised.h5ad",
    # Ex 1.5
    "https://cellgeni.cog.sanger.ac.uk/heartcellatlas/visium/OCT/HCAHeartST10659160.h5ad",
    "https://cellgeni.cog.sanger.ac.uk/heartcellatlas/visium/OCT/HCAHeartST13228103.h5ad",
]

for url in urls:
    print(f"Downloading {url}...")
    urlretrieve(url, DATA_DIR / Path(url).name)

### Exercise 1.1: Describe the UMAPs in the paper (5 points)

Please read the [paper](https://www.nature.com/articles/s41586-023-06311-1) and describe what is shown in Figure 1c) and 1d)?

a) what kind of data is the UMAP built from?

b) which cells were selected for visualization?

c) cells of which cell type are highlighted in panels c-d?



_Please insert your solution here_

### Exercise 1.2: Reproduce the UMAPs from the paper (2 points)
Reproduce the plot in Figure 1c) and 1d) using data in `AVN_aCMs_lognormalised.h5ad` and `SAN_aCMs_lognormalised.h5ad`. The celltype annotation is contained in the `obs` data frame of these anndata objects.

In [ ]:
## Solution



### Exercise 1.3: cell type deconvolution using cell2location (10 points)
Please read the cell2location [paper](https://www.nature.com/articles/s41587-021-01139-4) and the cell2location [documentation](https://cell2location.readthedocs.io/en/latest/index.html).

a) what is the input to cell2location?

b) what are the key steps in cell2location and which functions in the code do they correspond to?

c) what is the output of cell2location?

_Please insert your solution here_

### Exercise 1.4: cell type signature for spatial deconvolution (12 points)

Now we turn to the spot based spatial transcriptomics data that was generated using the Visum technology. This data was analysed using cell2location (see introductory talk).

We followed the code of the publication to learn the cell type signatures of the snRNA-seq data. The code is available here:
https://github.com/Teichlab/HCA_Heart_ver2/blob/main/Visium/1_cell2location/reference-signature_revision_SAN-AVN.ipynb

The following code chunk downloads and extracts the data.


In [ ]:
url = "https://hmgubox2.helmholtz-muenchen.de/index.php/s/XPN7fy5GQY3aZ6z/download"
urlretrieve(url, DATA_DIR / 'cell2location.zip')
with zipfile.ZipFile(DATA_DIR / "cell2location.zip", 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR)

The output files `sc.h5ad`, `model.pt`, `inf_aver.csv` are available here:
`DATA_DIR / "cell2location" / "reference-signatures" / <"AVN"/"SAN">`. When you work on google colab, you can see the files when you click on the files folder icon on the left.

**Our goal now is to verify the cell type signatures:**

a) Find the top 10 DE genes for the follwing 3 groups in AVN: P cells, bundle cells, and other cells.

b) Generate a dotplot of the marker genes over the 3 groups.

c) Based on the dotplot select 10 marker genes that are specific for 1 (or 2) of the 3 groups.

d) Create a heatmap or a table to visualize and check the `means_per_cluster_mu_fg` of the different states in AVN and discuss if the learned signatures fit to the marker expressions.

In [ ]:
# solution a-b

In [ ]:
# solution c

In [ ]:
# solution d


### Exercise 1.5: Reproduce figure 1f) (6 points)
Using these celltype signatures, cell2location has been applied to the Visium data sets in `HCAHeartST10659160.h5ad` for the first (left) panel of figure 1f) and `HCAHeartST13228103.h5ad` for the second and third panel of figure 1f). In each file, the `obsm` data frame contains the results of the cell2location deconvolution. Please refer to the [documentation](https://cell2location.readthedocs.io/en/latest/cell2location.html) of the `export_posterior` function to identify the relevant column that carries the information on the mean abundance per cell type.

Copy the abundance estimates of the cell types shown in the panels of Figure 1f) as new columns of the anndata `obs` data frame.

Identify the scanpy plotting function that is best suited for visualization of spatial data from the [documentation](https://scanpy.readthedocs.io/en/latest/api/plotting.html).

a) Use this function to plot the spatial distribution of the cell type abundance estimates created by cell2location in the three leftmost panels of figure 1f).

b) Based on your three plots and the [paper](https://www.nature.com/articles/s41586-023-06311-1): explain which of the cell types of the cardiac conduction system are localized in which physiological regions?

In [ ]:
# solution

## Exercise 2: spatial statistics on cell type resolved data

In this part of the exercise you will look at cell type resolved data from human heart tissue that was profiled on the 10X genomics Xenium platform.

In [ ]:
url_dict = {
    # Ex 2
    "cell_feature_matrix.h5": "https://hmgubox2.helmholtz-muenchen.de/index.php/s/6mPbYEb53xR3mKy/download",
    "cells_annotated.csv": "https://hmgubox2.helmholtz-muenchen.de/index.php/s/NiaBtWAEjrDDN8g/download",
}

for file in url_dict.keys():
    print(f"Downloading {file}...")
    urlretrieve(url_dict[file], DATA_DIR / file)

The next code chunk loads the data and plots the cell type annotation on the spatial coordinates. The cell types are annotated as:

| Annotation | Cell type |
|------------|-----------|
| CM | cardiomyocytes |
| EC | endothelial cells |
| FB | fibroblasts |
| Leuko | leukocytes |
| NC | neuronal cells |
| PC | pericytes |
| SMC | smooth muscle cells |

In [ ]:
adata = sc.read_10x_h5(filename = "%s/cell_feature_matrix.h5" % DATA_DIR)
cells = pd.read_csv("%s/cells_annotated.csv" % DATA_DIR)
adata = adata[cells.index]
adata.obs = cells.copy()
adata.obsm["spatial"] = adata.obs[["y_centroid", "x_centroid"]].copy().to_numpy()

sq.pl.spatial_scatter(
    adata, color="annotation", shape=None, figsize=(16, 5)
)

### Exercise 2.1: Identify neighborhood enrichment (5 points)
In this exercise the aim is to identfy whether specific cell types are preferentially localized next to cells of specific other cell types.

a) Check out the squidpy [Xenium tutorial](https://squidpy.readthedocs.io/en/stable/notebooks/tutorials/tutorial_xenium.html) to run the neighborhood enrichment analysis using the anndata object that we just loaded and `cluster_key="annotation"`. Create a heatmap plot and describe the result:
 - what does the color code indicate?
 - what is the interpretation of positive and negative values?
 - which is the highest value that you observe and how do you interpret it?

b) Plot the spatial distribution of the three cell types that are most frequent in vessel walls. What do you observe? How is this related to the results of part a)?


In [ ]:
# solution a

In [ ]:
# solution b

### Exercise 2.2: co-occurrence analysis (5 points)
A more general analysis of co-occurence patterns can be performed when not only the direct neighbors but also cells at greater distances are considered.

a) Check out the squidpy [Xenium tutorial](https://squidpy.readthedocs.io/en/stable/notebooks/tutorials/tutorial_xenium.html) to run the co-occurrence analysis using the anndata object that we just loaded and `cluster_key="annotation"`. Focus this analysis on smooth muscle cells (SMC).

b) Create the co-occurence plot for SMCs. Describe the plot: what is shown on each of the axes? How do you interprete the result for SMCs? Which tissue structure could explain this observation?

_Note_: you can speed up the computations by restricting the analyses to a smaller patch on the image, for example by subsetting the ann data object with this expression `(adata.obs["x_centroid"] > 4000) & (adata.obs["x_centroid"] < 6000) & (adata.obs["y_centroid"] > 5000) & (adata.obs["y_centroid"] < 10000)`

In [ ]:
# solution a

In [ ]:
# solution b

### Exercise 2.3: Identify genes with spatial gene expression (5 points)

Use the squidpy function [`squidpy.gr.autocorr`](https://squidpy.readthedocs.io/en/stable/api/squidpy.gr.spatial_autocorr.html) to identify three different types  spatially patterned genes:

a) Find the top 3 genes with the highest Moran's I

b) Find the top 3 genes with the highest Moran's I after filtering for genes that are expressed in at least 10% of cells

c) Find the top 3 genes with the lowest Moran's I after filtering for genes that are expressed in at least 10% of cells

Use the squidpy function [`squidpy.pl.spatial_scatter`](https://squidpy.readthedocs.io/en/stable/api/squidpy.pl.spatial_scatter.html) to plot the spatial expression pattern of each of the top 3 genes from a)-c).


In [ ]:
# solution